# Phylogenetic comparative methods

### using [BayesTraits](http://www.evolution.rdg.ac.uk/BayesTraits.html) 3.0.1

#### Motif set: (Sp-D, Sp-L, Maze)
#### Top 10 major fish orders

In [ ]:
version

In [ ]:
library(tidyverse)
library(ggpubr)

packageVersion("tidyverse")
packageVersion("ggpubr")

source("PCM_misc.R")

In [ ]:
# num_gen: the number of genera
# num_gen2: the number of genera for which trait data are available
# num_gen_Maze: the number of genera in which Maze motifs occur

top10odrs <- read_csv("top10odrs.csv", col_types = "ciii")
top10odrs

In [ ]:
get_MgLh <- function(target_order, ptns, model, hp_exp1, hp_exp2, rep) {
    fbase <- str_c(target_order, "state", ptns, sep="_")
    modelpar <- str_c(model, "rjMCMC_SCT_HPexp", hp_exp1, hp_exp2, "rep", rep, sep="_")
    stones_file <- str_c("./logs/", fbase, "_", modelpar, ".Stones.txt", sep = "")
    
    if (! file.exists(stones_file)) {
        print(str_c("file not found: ", stones_file))
        return(NA_real_)
    }
    
    MgLh <- NA_real_
    tryCatch({
        df_MgLh <- read_tsv(stones_file, skip = 107, col_names = FALSE, col_types="cd")
        MgLh <- df_MgLh[[1, 2]]
        }, error=function(e){}
    )
    return(MgLh)
}

get_df_MCMC <- function(target_order, ptns, model, hp_exp1, hp_exp2, rep) {
    fbase <- str_c(target_order, "state", ptns, sep="_")
    modelpar <- str_c(model, "rjMCMC_SCT_HPexp", hp_exp1, hp_exp2, "rep", rep, sep="_")
    log_file <- str_c("./logs/", fbase, "_", modelpar, ".Log.txt", sep = "")
    
    if (! file.exists(log_file)) {
        print(str_c("file not found: ", log_file))
        return(NA_real_)
    }

    colsp <- cols(.default = col_double(),
                  `Model string` = col_character())

    df_MCMC <- NA
    tryCatch({
        start_line <- grep("^Iteration\tLh", read_lines(log_file))
        # print(str_c(start_line, " lines skipped"))
        df_MCMC <- read_tsv(log_file,
                            skip = (start_line - 1),
                            col_types = colsp)
        df_MCMC <- df_MCMC %>%
            select(-starts_with("X"))
        
        }, error=function(e){ }
    )
    
    return(df_MCMC)
}

get_mean_rates <- function(target_order, ptns, model, rep, row, column) {
    df <- get_df_MCMC(target_order, ptns, model, 0, 10, rep)
    return(rateMatMean(df)[row, column])
}


In [ ]:
ptns <- "Sp_D_Sp_L_Maze"
hp_exp1 <- 0
hp_exp2 <- 10
models = c("IND", "indZ", "DEP")

get_df_model_rep <- function(model, rep) {
    top10odrs %>%
    mutate(
        MgLh = map_dbl(order, get_MgLh, ptns = ptns, model = model,
                       hp_exp1 = hp_exp1, hp_exp2 = hp_exp2,
                       rep = rep),
        ptns = ptns,
        model = model,
        rep = rep
    )
}

df_1 <- bind_rows(map(models, get_df_model_rep, 1))
df_2 <- bind_rows(map(models, get_df_model_rep, 2))
df_3 <- bind_rows(map(models, get_df_model_rep, 3))
df_odrs <- bind_rows(df_1, df_2, df_3)

df_odrs %>% head(30)

In [ ]:
df_rates_odrs <- df_odrs %>%
    mutate(rates = pmap(list(order, ptns, model, rep), get_mean_rates))

In [ ]:
df_rates <- df_rates_odrs %>%
    mutate(
        q14 = map_dbl(rates, ~ .x[1, 4]),
        q26 = map_dbl(rates, ~ .x[2, 6]),
        q37 = map_dbl(rates, ~ .x[3, 7]),
        q58 = map_dbl(rates, ~ .x[5, 8]),
        q41 = map_dbl(rates, ~ .x[4, 1]),
        q62 = map_dbl(rates, ~ .x[6, 2]),
        q73 = map_dbl(rates, ~ .x[7, 3]),
        q85 = map_dbl(rates, ~ .x[8, 5])
    ) %>%
    select(order, num_gen:MgLh, model, rep, q14:q85) %>%
    pivot_longer(cols = c(q14, q26, q37, q58, q41, q62, q73, q85), names_to = "qij", values_to = "rate")

In [ ]:
df_rates

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 5)

mgain <- df_rates %>%
    filter(model == "DEP") %>%
    group_by(order, qij) %>%
    summarize(mean_rate = mean(rate), num_gen2 = mean(num_gen2)) %>%
    ungroup() %>%
    pivot_wider(names_from = qij, values_from = mean_rate) %>%
    drop_na() %>%
    arrange(desc(num_gen2)) %>%
    pivot_longer(cols = c(q14, q26, q37, q58), names_to = "qij", values_to = "mean_rate") %>%

    ggplot(aes(x = qij, y = mean_rate)) +
        geom_line(aes(color = fct_reorder(order, desc(num_gen2)), group = order), size = 1) +
        geom_point(aes(color = order), size = 3) +
        scale_x_discrete(labels = c("a", "b", "c", "d"), expand = c(0.1, 0.1)) +
        scale_y_continuous(breaks = seq(0, 8, by = 2)) +
        theme_classic() +
        theme(axis.text.x = element_text(face = "italic", size = 22, color = "black"),
              axis.text.y = element_text(size = 12, color = "black"),
              axis.title.y = element_text(size = 18),
              legend.text = element_text(size = 14),
              legend.key.size = unit(1, "line"),
              plot.margin = unit(c(1, 0.7, 0, 1), "lines")) +
        labs(x = "", y = "Transition rate (Maze gain)", color = "")

mloss <- df_rates %>%
    filter(model == "DEP") %>%
    group_by(order, qij) %>%
    summarize(mean_rate = mean(rate), num_gen2 = mean(num_gen2)) %>%
    ungroup() %>%
    pivot_wider(names_from = qij, values_from = mean_rate) %>%
    drop_na() %>%
    arrange(desc(num_gen2)) %>%
    pivot_longer(cols = c(q41, q62, q73, q85), names_to = "qij", values_to = "mean_rate") %>%

    ggplot(aes(x = qij, y = mean_rate)) +
        geom_line(aes(color = fct_reorder(order, desc(num_gen2)), group = order), size = 1) +
        geom_point(aes(color = order), size = 3) +
        scale_x_discrete(labels = c("e", "f", "g", "h"), expand = c(0.1, 0.1)) +
        scale_y_continuous(breaks = seq(0, 8, by = 2)) +
        theme_classic() +
        theme(axis.text.x = element_text(face = "italic", size = 22, color = "black"),
              axis.text.y = element_text(size = 12, color = "black"),
              axis.title.y = element_text(size = 18),
              legend.text = element_text(size = 14),
              legend.key.size = unit(1, "line"),
              plot.margin = unit(c(1, 1, 0, 0.7), "lines")) +
        labs(x = "", y = "Transition rate (Maze loss)", color = "")

plots <- ggarrange(mgain, mloss,
          ncol = 2,
          common.legend = TRUE, legend = "bottom")

plots

plots %>%
    ggsave(file = "Maze_gain_loss_rates.pdf", width = 7, height = 5)

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 6)

df_rates_odrs %>%
    filter(order == "Perciformes",
           model == "IND",
           rep == 1) %>%
    pull(rates) %>%
    .[[1]] -> mat

plotRatesInd(mat, main = "Perciformes (IND model)", sub = "(Sp_D, Sp_L, Maze)")


In [ ]:
options(repr.plot.width = 6, repr.plot.height = 6)

df_rates_odrs %>%
    filter(order == "Perciformes",
           model == "DEP",
           rep == 1) %>%
    pull(rates) %>%
    .[[1]] -> mat

plotRates(mat, main = "Perciformes (DEP model)", sub = "(Sp_D, Sp_L, Maze)")


In [ ]:
options(repr.plot.width = 6, repr.plot.height = 6)

df_rates_odrs %>%
    filter(order == "Perciformes",
           model == "indZ",
           rep == 1) %>%
    pull(rates) %>%
    .[[1]] -> mat

plotRatesIndZ(mat, main = "Perciformes (indZ model)", sub = "(Sp_D, Sp_L, Maze)")

### Bayes factor (DEP - IND)

In [ ]:
df_rates_full <- df_rates_odrs %>%
    mutate(
        q12 = map_dbl(rates, ~ .x[1, 2]),
        q13 = map_dbl(rates, ~ .x[1, 3]),
        q14 = map_dbl(rates, ~ .x[1, 4]),
        q21 = map_dbl(rates, ~ .x[2, 1]),
        q25 = map_dbl(rates, ~ .x[2, 5]),
        q26 = map_dbl(rates, ~ .x[2, 6]),
        q31 = map_dbl(rates, ~ .x[3, 1]),
        q35 = map_dbl(rates, ~ .x[3, 5]),
        q37 = map_dbl(rates, ~ .x[3, 7]),
        q41 = map_dbl(rates, ~ .x[4, 1]),
        q46 = map_dbl(rates, ~ .x[4, 6]),
        q47 = map_dbl(rates, ~ .x[4, 7]),
        q52 = map_dbl(rates, ~ .x[5, 2]),
        q53 = map_dbl(rates, ~ .x[5, 3]),
        q58 = map_dbl(rates, ~ .x[5, 8]),
        q62 = map_dbl(rates, ~ .x[6, 2]),
        q64 = map_dbl(rates, ~ .x[6, 4]),
        q68 = map_dbl(rates, ~ .x[6, 8]),
        q73 = map_dbl(rates, ~ .x[7, 3]),
        q74 = map_dbl(rates, ~ .x[7, 4]),
        q78 = map_dbl(rates, ~ .x[7, 8]),
        q85 = map_dbl(rates, ~ .x[8, 5]),
        q86 = map_dbl(rates, ~ .x[8, 6]),
        q87 = map_dbl(rates, ~ .x[8, 7])
    ) %>%
    select(order, Ngen = num_gen2, NgenMaze = num_gen_Maze, MgLh, model, rep, q12:q87)

In [ ]:
df_MgLh_BF_DEP_IND <- df_rates_full %>%
    select(order:rep) %>%
    pivot_wider(names_from = model, names_prefix = "MgLh_", values_from = MgLh) %>%
    select(order, Ngen, NgenMaze, rep, MgLh_IND, MgLh_DEP) %>%
    arrange(desc(Ngen), desc(NgenMaze), rep) %>%
    mutate(
        logBF_DEP_IND = 2 * (MgLh_DEP - MgLh_IND)
    ) %>%

    mutate(evidence = case_when(
        logBF_DEP_IND > 10 ~ "***",
        (5 < logBF_DEP_IND & logBF_DEP_IND <= 10) ~ "**",
        (2 < logBF_DEP_IND & logBF_DEP_IND <= 5) ~ "*",
        logBF_DEP_IND <= 2 ~ "")) %>%

    mutate(MgLh_IND = format(MgLh_IND, digits = 3, nsmall = 2, trim = TRUE),
           MgLh_DEP = format(MgLh_DEP, digits = 3, nsmall = 2, trim = TRUE),
           logBF_DEP_IND = str_c(format(logBF_DEP_IND, digits = 1, nsmall = 2, trim = TRUE), " ", evidence))

df_MgLh_BF_DEP_IND